In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import kalepy as kale

from holodeck import plot, detstats, utils, cosmo
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo
from holodeck import extensions as holo_extensions


In [ ]:
fobs_cents, fobs_edges = utils.pta_freqs()
fobs_orb_cents, fobs_orb_edges = fobs_cents/2.0, fobs_edges/2.0
sam = holo.sams.Semi_Analytic_Model()
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)
realizer = holo_extensions.Realizer_SAM(fobs_orb_edges=fobs_edges/2.0,
                                        sam=sam, hard=hard)

In [ ]:
redz_final, diff_num = holo.sams.cyutils.dynamic_binary_number_at_fobs(
        fobs_orb_cents, sam, hard, cosmo
    )

edges = [sam.mtot, sam.mrat, sam.redz, fobs_orb_edges]
number = holo.sams.cyutils.integrate_differential_number_3dx1d(edges, diff_num) # fractional number per bin



In [ ]:
print(holo.utils.stats(redz_final))
print(holo.utils.stats(sam.redz))

In [ ]:
samples = holo_extensions.get_samples_from_edges(edges, redz_final, number.shape)

In [ ]:
print(holo.utils.stats(samples[2]))

In [ ]:
nreals=3
names = ['mtot', 'mrat', 'redz', 'fobs']
number = number.flatten()
shape = (number.size, nreals)
weights = holo.gravwaves.poisson_as_needed(number[..., np.newaxis] * np.ones(shape)).reshape(shape)


In [ ]:
nonzero_samples = []
nonzero_weights = []
for rr in range(nreals):
    nonzero = (weights[:,rr]!=0) & (samples[2]>0)
    print(nonzero)
    mtot = samples[0][nonzero]
    mrat = samples[1][nonzero]
    redz = samples[2][nonzero]
    fobs = samples[3][nonzero]
    nonzero_samples.append([mtot, mrat, redz, fobs])
    nonzero_weights.append(weights[:,rr][nonzero])

In [ ]:
print(holo.utils.stats(redz))

In [ ]:
print(len(nonzero_samples[0][0]))
print(len(samples[0])/3)